## Import libraries

In [ ]:
from google.colab import drive
from pathlib import Path
from matplotlib import pyplot as plt 
import pandas as pd
import numpy as np
import time
import os
import csv
import concurrent.futures

## Utility functions

### Create annot and load descriptors

In [ ]:
def create_annot(path):
  image_list = list(Path(path).glob('*/*.jpg'))
  # the identity name is in the path (the name of the parent directory)
  names_list = [i.parent.name for i in image_list]  # get the identity of each image
  # keep info in a pandas DataFrame
  annot = pd.DataFrame({'identity': names_list, 'image_path': image_list})
  return annot

def concatenate_annots(list_of_paths):
  concat_annot = pd.DataFrame()
  with concurrent.futures.ThreadPoolExecutor() as executor:
    annots = [executor.submit(create_annot, path) for path in list_of_paths]
    for annot in annots:
      new_annot = annot.result()
      concat_annot = concat_annot.append(new_annot, ignore_index = True)
    return concat_annot

In [ ]:
def load_descriptors(path):
  with open(path, 'rb') as file:
    return np.load(file)

def concatenate_descriptors(list_of_paths):
  concat_descriptors = None
  with concurrent.futures.ThreadPoolExecutor() as executor:
    descriptors = [executor.submit(load_descriptors, path) for path in list_of_paths]
    for descriptor in descriptors:
      new_descriptor = descriptor.result()
      if concat_descriptors is None:
        concat_descriptors = new_descriptor
      else:
        concat_descriptors = np.concatenate([concat_descriptors, new_descriptor])
    return concat_descriptors

### Create pivots

In [ ]:
def generate_pivots(descriptors, n, strategy="rnd"):
  if strategy == "kMED":
    kmedoids = sklearn_extra.cluster.KMedoids(n_clusters=n).fit(descriptors)
    return kmedoids.cluster_centers_
  if strategy != "rnd":
    print(strategy, "was not implemented. Random pivots were returned")
  pivots_id = np.random.choice(np.arange(len(descriptors)), size=n)
  return descriptors[pivots_id]

def generate_list_of_pivots(descriptors, t, n, strategy="rnd"):
  list_of_pivots = []
  with concurrent.futures.ThreadPoolExecutor() as executor:
    pivots = [executor.submit(generate_pivots, descriptors, n, strategy) for i in range(t)]
    for pivot in concurrent.futures.as_completed(pivots):
      new_pivot = pivot.result()
      list_of_pivots.append(new_pivot)
    return list_of_pivots

### Save test results

In [ ]:
def save_results(dir, file_name, results):
  with open(os.path.join(dir, file_name +".csv"), 'w') as f:
    writer = csv.writer(f)
    # write the header
    writer.writerow(["CLASS", "AP", "QUERY TIME"])
    # write the data
    for r in results:
      writer.writerow(r) 

## Test Performance

In [ ]:
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


### Create annot and load descriptors for the database

In [ ]:
db_annot = concatenate_annots(['/content/drive/MyDrive/CV_Birds/train', '/content/drive/MyDrive/CV_Birds/mirflickr25k'])
db_annot

,identity,image_path
0,AFRICAN CROWNED CRANE,/content/drive/MyDrive/CV_Birds/train/AFRICAN ...
1,AFRICAN CROWNED CRANE,/content/drive/MyDrive/CV_Birds/train/AFRICAN ...
2,AFRICAN CROWNED CRANE,/content/drive/MyDrive/CV_Birds/train/AFRICAN ...
3,AFRICAN CROWNED CRANE,/content/drive/MyDrive/CV_Birds/train/AFRICAN ...
4,AFRICAN CROWNED CRANE,/content/drive/MyDrive/CV_Birds/train/AFRICAN ...
...,...,...
72327,mirflickr,/content/drive/MyDrive/CV_Birds/mirflickr25k/m...
72328,mirflickr,/content/drive/MyDrive/CV_Birds/mirflickr25k/m...
72329,mirflickr,/content/drive/MyDrive/CV_Birds/mirflickr25k/m...
72330,mirflickr,/content/drive/MyDrive/CV_Birds/mirflickr25k/m...


In [ ]:
db_descriptors = concatenate_descriptors(['/content/drive/MyDrive/CV_Birds/features/training/AutoEncoder/512to128withPace64.npy','/content/drive/MyDrive/CV_Birds/features/distractor/AutoEncoder/512to128withPace64.npy'])
db_descriptors.shape

(72332, 128)

### Create annot and load descriptors for the test set

In [ ]:
query_annot = create_annot('/content/drive/MyDrive/CV_Birds/test')
query_annot

,identity,image_path
0,AFRICAN CROWNED CRANE,/content/drive/MyDrive/CV_Birds/test/AFRICAN C...
1,AFRICAN CROWNED CRANE,/content/drive/MyDrive/CV_Birds/test/AFRICAN C...
2,AFRICAN CROWNED CRANE,/content/drive/MyDrive/CV_Birds/test/AFRICAN C...
3,AFRICAN CROWNED CRANE,/content/drive/MyDrive/CV_Birds/test/AFRICAN C...
4,AFRICAN CROWNED CRANE,/content/drive/MyDrive/CV_Birds/test/AFRICAN C...
...,...,...
1620,YELLOW HEADED BLACKBIRD,/content/drive/MyDrive/CV_Birds/test/YELLOW HE...
1621,YELLOW HEADED BLACKBIRD,/content/drive/MyDrive/CV_Birds/test/YELLOW HE...
1622,YELLOW HEADED BLACKBIRD,/content/drive/MyDrive/CV_Birds/test/YELLOW HE...
1623,YELLOW HEADED BLACKBIRD,/content/drive/MyDrive/CV_Birds/test/YELLOW HE...


In [ ]:
query_descriptors = load_descriptors('/content/drive/MyDrive/CV_Birds/features/test/AutoEncoder/512to128withPace64.npy')
query_descriptors.shape

(1625, 128)

To run our tests we select only the first image of each species within the test set. Please note that within the test set we have 5 images per species.

In [ ]:
queries_indexes = [x for x in range(325*5) if x%5 == 0]

### Create PP-Index

In [ ]:
def get_descriptor_from_id(id_object):
  return db_descriptors[id_object]

In [ ]:
%cd "/content/drive/MyDrive/CV_Birds/Notebooks/PP-Index"
%run PPIndex.ipynb

/content/drive/.shortcut-targets-by-id/1rI5YNBuaSlCB__w522WEkHjw-nFuvIo0/CV_Birds/Notebooks/PP-Index


In [ ]:
# generate pivots
pivots = generate_pivots(db_descriptors, 40, "rnd")

In [ ]:
# cosine tree
cosine_tree = PrefixTree(pivots, length=3, distance_metric='cosine', base_directory="/content/drive/MyDrive/CV_Birds/indexes/fine_tuning/tree/cosine", tree_file='tree_structure')
if cosine_tree.is_empty():
  cosine_tree.insert_objects_into_tree(range(len(db_descriptors)))
  cosine_tree.save()

[WARNING]: Tree is still empty
Inserting objects into tree_structure ...
--- Insertion into tree tree_structure completed: 504.566 seconds ---


In [ ]:
# euclidean tree
euclidean_tree = PrefixTree(pivots, length=3, distance_metric='euclidean', base_directory="/content/drive/MyDrive/CV_Birds/indexes/fine_tuning/tree/euclidean", tree_file='tree_structure')
if euclidean_tree.is_empty():
  euclidean_tree.insert_objects_into_tree(range(len(db_descriptors)))
  euclidean_tree.save()

Tree retrieved from disk


### Compute mAP

In [ ]:
birds_db = db_annot.loc[db_annot['identity'] != 'mirflickr']
counts = birds_db.groupby('identity').count()
print("Minimum number of images per species:", int(counts.min()))
print("Maximum number of images per species:", int(counts.max()))
print("Average number of images:", float(counts.sum()/325))

Minimum number of images per species: 116
Maximum number of images per species: 249
Average number of images: 145.63692307692307


Since at most we have 249 images per species, we use $n=250$.

In [ ]:
n = 250

The formula for Average Precision is the following:

> $AP@n=\frac{1}{GTP}\sum_{k=1}^{n}P@k×rel@k$

where $GTP$ refers to the total number of ground truth positives, $n$ refers to the total number of images we are interested in, $P@k$ refers to the precision@k and $rel@k$ is a relevance function. 

The relevance function is an indicator function which equals 1 if the document at rank $k$ is relevant and equals to 0 otherwise.

In [ ]:
def compute_ap(query_index, retrieved_ids):
  query_identity = query_annot['identity'][query_index]
  print(query_index//5, query_identity)
  GTP = len(db_annot.loc[db_annot['identity'] == query_identity])
  relevant = 0
  precision_summation = 0
  for k, id in enumerate(retrieved_ids):
    if db_annot['identity'][id] == query_identity: # relevant result
      relevant = relevant + 1
      precision_at_k = relevant/(k+1)
      precision_summation = precision_summation + precision_at_k
  return (query_identity, precision_summation/GTP)

For each query, $Q$, we can calculate a corresponding $AP$. Then, the $mAP$ is simply the mean of all the queries that were made.
> $mAP = \frac{1}{N}\sum_{i=1}^{N}AP_i$

In our case, $N=325$ (one query per species)

#### Simple tree

##### Cosine

In [ ]:
def cosine_tree_queries(query_index, n):
  start_time = time.time()
  ids, distances = cosine_tree.find_nearest_neighbors(query_descriptors[query_index], n)
  end_time = time.time()
  ids = ids.tolist()
  return compute_ap(query_index, ids) + (end_time - start_time,)

In [ ]:
aps = []
for query_index in queries_indexes:
  aps.append(cosine_tree_queries(query_index, n))

0 AFRICAN CROWNED CRANE
1 AFRICAN FIREFINCH
2 ALBATROSS
3 ALEXANDRINE PARAKEET
4 AMERICAN AVOCET
5 AMERICAN BITTERN
6 AMERICAN COOT
7 AMERICAN GOLDFINCH
8 AMERICAN KESTREL
9 AMERICAN PIPIT
10 AMERICAN REDSTART
11 ANHINGA
12 ANNAS HUMMINGBIRD
13 ANTBIRD
14 ARARIPE MANAKIN
15 ASIAN CRESTED IBIS
16 BALD EAGLE
17 BALD IBIS
18 BALI STARLING
19 BALTIMORE ORIOLE
20 BANANAQUIT
21 BANDED BROADBILL
22 BANDED PITA
23 BAR-TAILED GODWIT
24 BARN OWL
25 BARN SWALLOW
26 BARRED PUFFBIRD
27 BAY-BREASTED WARBLER
28 BEARDED BARBET
29 BEARDED BELLBIRD
30 BEARDED REEDLING
31 BELTED KINGFISHER
32 BIRD OF PARADISE
33 BLACK & YELLOW bROADBILL
34 BLACK BAZA
35 BLACK FRANCOLIN
36 BLACK SKIMMER
37 BLACK SWAN
38 BLACK TAIL CRAKE
39 BLACK THROATED BUSHTIT
40 BLACK THROATED WARBLER
41 BLACK VULTURE
42 BLACK-CAPPED CHICKADEE
43 BLACK-NECKED GREBE
44 BLACK-THROATED SPARROW
45 BLACKBURNIAM WARBLER
46 BLONDE CRESTED WOODPECKER
47 BLUE COAU
48 BLUE GROUSE
49 BLUE HERON
50 BLUE THROATED TOUCANET
51 BOBOLINK
52 BORNEAN BRI

In [ ]:
aps

[('AFRICAN CROWNED CRANE', 0.9765713660322425, 0.9975061416625977),
 ('AFRICAN FIREFINCH', 0.2840814570573726, 1.289513111114502),
 ('ALBATROSS', 0.34610989002070847, 0.90541672706604),
 ('ALEXANDRINE PARAKEET', 0.20124386661338634, 1.245220422744751),
 ('AMERICAN AVOCET', 0.5080232156782533, 1.5991156101226807),
 ('AMERICAN BITTERN', 0.9727122992382451, 1.6156070232391357),
 ('AMERICAN COOT', 0.8164556962025317, 1.6403694152832031),
 ('AMERICAN GOLDFINCH', 0.7428055299936714, 1.5388872623443604),
 ('AMERICAN KESTREL', 0.9250947872435533, 1.9063408374786377),
 ('AMERICAN PIPIT', 0.42246827732203995, 1.8635852336883545),
 ('AMERICAN REDSTART', 0.14162721997005007, 1.9879112243652344),
 ('ANHINGA', 0.32266991050341737, 1.3238680362701416),
 ('ANNAS HUMMINGBIRD', 0.4360890162653829, 1.6461231708526611),
 ('ANTBIRD', 0.09321758547617888, 1.858332872390747),
 ('ARARIPE MANAKIN', 0.16774193548387098, 1.7754337787628174),
 ('ASIAN CRESTED IBIS', 0.17048995918585572, 1.1897695064544678),
 ('BA

In [ ]:
ap_at_n = np.array([ap[1] for ap in aps])
query_time = np.array(([ap[2] for ap in aps]))

In [ ]:
mAP_at_n = np.mean(ap_at_n, axis=0)
avg_query_time = np.mean(query_time, axis=0)
print("mAP:", mAP_at_n)
print("avg. query time: ", avg_query_time)

mAP: 0.3282295118084191
avg. query time:  1.5333857939793514


In [ ]:
save_results('/content/drive/MyDrive/CV_Birds/performance/fine_tuning/index/AutoEncoder', 'AE_FT_tree_cosine_results', aps)

##### Euclidean

In [ ]:
def euclidean_tree_queries(query_index, n):
  start_time = time.time()
  ids, distances = euclidean_tree.find_nearest_neighbors(query_descriptors[query_index], n)
  end_time = time.time()
  ids = ids.tolist()
  return compute_ap(query_index, ids) + (end_time - start_time,)

In [ ]:
aps = []
for query_index in queries_indexes:
  aps.append(euclidean_tree_queries(query_index, n))

0 AFRICAN CROWNED CRANE
1 AFRICAN FIREFINCH
2 ALBATROSS
3 ALEXANDRINE PARAKEET
4 AMERICAN AVOCET
5 AMERICAN BITTERN
6 AMERICAN COOT
7 AMERICAN GOLDFINCH
8 AMERICAN KESTREL
9 AMERICAN PIPIT
10 AMERICAN REDSTART
11 ANHINGA
12 ANNAS HUMMINGBIRD
13 ANTBIRD
14 ARARIPE MANAKIN
15 ASIAN CRESTED IBIS
16 BALD EAGLE
17 BALD IBIS
18 BALI STARLING
19 BALTIMORE ORIOLE
20 BANANAQUIT
21 BANDED BROADBILL
22 BANDED PITA
23 BAR-TAILED GODWIT
24 BARN OWL
25 BARN SWALLOW
26 BARRED PUFFBIRD
27 BAY-BREASTED WARBLER
28 BEARDED BARBET
29 BEARDED BELLBIRD
30 BEARDED REEDLING
31 BELTED KINGFISHER
32 BIRD OF PARADISE
33 BLACK & YELLOW bROADBILL
34 BLACK BAZA
35 BLACK FRANCOLIN
36 BLACK SKIMMER
37 BLACK SWAN
38 BLACK TAIL CRAKE
39 BLACK THROATED BUSHTIT
40 BLACK THROATED WARBLER
41 BLACK VULTURE
42 BLACK-CAPPED CHICKADEE
43 BLACK-NECKED GREBE
44 BLACK-THROATED SPARROW
45 BLACKBURNIAM WARBLER
46 BLONDE CRESTED WOODPECKER
47 BLUE COAU
48 BLUE GROUSE
49 BLUE HERON
50 BLUE THROATED TOUCANET
51 BOBOLINK
52 BORNEAN BRI

In [ ]:
aps

[('AFRICAN CROWNED CRANE', 0.10289783617721361, 1.163726568222046),
 ('AFRICAN FIREFINCH', 0.07077922077922078, 1.4923241138458252),
 ('ALBATROSS', 0.516447028383978, 0.8230712413787842),
 ('ALEXANDRINE PARAKEET', 0.3284641469311701, 1.1224830150604248),
 ('AMERICAN AVOCET', 0.4106181025628968, 1.775561809539795),
 ('AMERICAN BITTERN', 0.9614809634613626, 1.6276917457580566),
 ('AMERICAN COOT', 0.6857464697200188, 1.38580322265625),
 ('AMERICAN GOLDFINCH', 0.545943399329572, 1.147714614868164),
 ('AMERICAN KESTREL', 0.5779001637506113, 1.517453670501709),
 ('AMERICAN PIPIT', 0.18867445403758254, 1.8542346954345703),
 ('AMERICAN REDSTART', 0.07926266837943743, 1.309168815612793),
 ('ANHINGA', 0.10019662780254494, 1.2638890743255615),
 ('ANNAS HUMMINGBIRD', 0.2283617551111721, 1.3015711307525635),
 ('ANTBIRD', 0.0, 1.2964463233947754),
 ('ARARIPE MANAKIN', 0.2438047558930411, 1.312009572982788),
 ('ASIAN CRESTED IBIS', 0.1027016102418875, 1.2955076694488525),
 ('BALD EAGLE', 0.5556794627

In [ ]:
ap_at_n = np.array([ap[1] for ap in aps])
query_time = np.array(([ap[2] for ap in aps]))

In [ ]:
mAP_at_n = np.mean(ap_at_n, axis=0)
avg_query_time = np.mean(query_time, axis=0)
print("mAP:", mAP_at_n)
print("avg. query time: ", avg_query_time)

mAP: 0.22129956230240005
avg. query time:  1.313885206075815


In [ ]:
save_results('/content/drive/MyDrive/CV_Birds/performance/fine_tuning/index/AutoEncoder', 'AE_FT_tree_euclidean_results', aps)

#### Tree with query perturbation

##### Cosine

In [ ]:
def cosine_pert_tree_queries(query_index, n):
  start_time = time.time()
  ids, distances = cosine_tree.find_nearest_neighbors_with_query_perturbation(query_descriptors[query_index], n, perturbations=3)
  end_time = time.time()
  ids = ids.tolist()
  return compute_ap(query_index, ids) + (end_time - start_time,)

In [ ]:
aps = []
for query_index in queries_indexes:
  aps.append(cosine_pert_tree_queries(query_index, n))

0 AFRICAN CROWNED CRANE
1 AFRICAN FIREFINCH
2 ALBATROSS
3 ALEXANDRINE PARAKEET
4 AMERICAN AVOCET
5 AMERICAN BITTERN
6 AMERICAN COOT
7 AMERICAN GOLDFINCH
8 AMERICAN KESTREL
9 AMERICAN PIPIT
10 AMERICAN REDSTART
11 ANHINGA
12 ANNAS HUMMINGBIRD
13 ANTBIRD
14 ARARIPE MANAKIN
15 ASIAN CRESTED IBIS
16 BALD EAGLE
17 BALD IBIS
18 BALI STARLING
19 BALTIMORE ORIOLE
20 BANANAQUIT
21 BANDED BROADBILL
22 BANDED PITA
23 BAR-TAILED GODWIT
24 BARN OWL
25 BARN SWALLOW
26 BARRED PUFFBIRD
27 BAY-BREASTED WARBLER
28 BEARDED BARBET
29 BEARDED BELLBIRD
30 BEARDED REEDLING
31 BELTED KINGFISHER
32 BIRD OF PARADISE
33 BLACK & YELLOW bROADBILL
34 BLACK BAZA
35 BLACK FRANCOLIN
36 BLACK SKIMMER
37 BLACK SWAN
38 BLACK TAIL CRAKE
39 BLACK THROATED BUSHTIT
40 BLACK THROATED WARBLER
41 BLACK VULTURE
42 BLACK-CAPPED CHICKADEE
43 BLACK-NECKED GREBE
44 BLACK-THROATED SPARROW
45 BLACKBURNIAM WARBLER
46 BLONDE CRESTED WOODPECKER
47 BLUE COAU
48 BLUE GROUSE
49 BLUE HERON
50 BLUE THROATED TOUCANET
51 BOBOLINK
52 BORNEAN BRI

In [ ]:
aps

[('AFRICAN CROWNED CRANE', 0.9761057368693203, 2.0760343074798584),
 ('AFRICAN FIREFINCH', 0.30899629810965, 2.239398717880249),
 ('ALBATROSS', 0.3577986540829709, 1.7794299125671387),
 ('ALEXANDRINE PARAKEET', 0.43022842334944306, 2.0393683910369873),
 ('AMERICAN AVOCET', 0.8433204971339865, 2.458096742630005),
 ('AMERICAN BITTERN', 0.9728397317755707, 2.5298213958740234),
 ('AMERICAN COOT', 0.8606677673821317, 2.580498218536377),
 ('AMERICAN GOLDFINCH', 0.7757577139443781, 2.387624979019165),
 ('AMERICAN KESTREL', 0.9244991906322075, 3.002105236053467),
 ('AMERICAN PIPIT', 0.49359416823041674, 2.356632709503174),
 ('AMERICAN REDSTART', 0.17331185100495547, 2.791818141937256),
 ('ANHINGA', 0.33698923598159575, 1.9802799224853516),
 ('ANNAS HUMMINGBIRD', 0.43561441299336784, 2.748687505722046),
 ('ANTBIRD', 0.13510166645805496, 2.849682331085205),
 ('ARARIPE MANAKIN', 0.19849399006562882, 2.6790719032287598),
 ('ASIAN CRESTED IBIS', 0.30571618907326836, 2.1154229640960693),
 ('BALD EAG

In [ ]:
ap_at_n = np.array([ap[1] for ap in aps])
query_time = np.array(([ap[2] for ap in aps]))

In [ ]:
mAP_at_n = np.mean(ap_at_n, axis=0)
avg_query_time = np.mean(query_time, axis=0)
print("mAP:", mAP_at_n)
print("avg. query time: ", avg_query_time)

mAP: 0.4475183617899105
avg. query time:  2.4459079911158637


In [ ]:
save_results('/content/drive/MyDrive/CV_Birds/performance/fine_tuning/index/AutoEncoder', 'AE_FT_pert_tree_cosine_results', aps)

##### Euclidean

In [ ]:
def euclidean_pert_tree_queries(query_index, n):
  start_time = time.time()
  ids, distances = euclidean_tree.find_nearest_neighbors_with_query_perturbation(query_descriptors[query_index], n, perturbations=3)
  end_time = time.time()
  ids = ids.tolist()
  return compute_ap(query_index, ids) + (end_time - start_time,)

In [ ]:
aps = []
for query_index in queries_indexes:
  aps.append(euclidean_pert_tree_queries(query_index, n))

0 AFRICAN CROWNED CRANE
1 AFRICAN FIREFINCH
2 ALBATROSS
3 ALEXANDRINE PARAKEET
4 AMERICAN AVOCET
5 AMERICAN BITTERN
6 AMERICAN COOT
7 AMERICAN GOLDFINCH
8 AMERICAN KESTREL
9 AMERICAN PIPIT
10 AMERICAN REDSTART
11 ANHINGA
12 ANNAS HUMMINGBIRD
13 ANTBIRD
14 ARARIPE MANAKIN
15 ASIAN CRESTED IBIS
16 BALD EAGLE
17 BALD IBIS
18 BALI STARLING
19 BALTIMORE ORIOLE
20 BANANAQUIT
21 BANDED BROADBILL
22 BANDED PITA
23 BAR-TAILED GODWIT
24 BARN OWL
25 BARN SWALLOW
26 BARRED PUFFBIRD
27 BAY-BREASTED WARBLER
28 BEARDED BARBET
29 BEARDED BELLBIRD
30 BEARDED REEDLING
31 BELTED KINGFISHER
32 BIRD OF PARADISE
33 BLACK & YELLOW bROADBILL
34 BLACK BAZA
35 BLACK FRANCOLIN
36 BLACK SKIMMER
37 BLACK SWAN
38 BLACK TAIL CRAKE
39 BLACK THROATED BUSHTIT
40 BLACK THROATED WARBLER
41 BLACK VULTURE
42 BLACK-CAPPED CHICKADEE
43 BLACK-NECKED GREBE
44 BLACK-THROATED SPARROW
45 BLACKBURNIAM WARBLER
46 BLONDE CRESTED WOODPECKER
47 BLUE COAU
48 BLUE GROUSE
49 BLUE HERON
50 BLUE THROATED TOUCANET
51 BOBOLINK
52 BORNEAN BRI

In [ ]:
aps

[('AFRICAN CROWNED CRANE', 0.16241630436684856, 1.5808601379394531),
 ('AFRICAN FIREFINCH', 0.22369966694559695, 1.9347615242004395),
 ('ALBATROSS', 0.5234689227759609, 1.3503568172454834),
 ('ALEXANDRINE PARAKEET', 0.5718723148250761, 1.6563971042633057),
 ('AMERICAN AVOCET', 0.4996798719079185, 2.2920281887054443),
 ('AMERICAN BITTERN', 0.9550746003567194, 2.135024309158325),
 ('AMERICAN COOT', 0.7493600954503158, 1.9284725189208984),
 ('AMERICAN GOLDFINCH', 0.6065784864328859, 1.9799120426177979),
 ('AMERICAN KESTREL', 0.5085851017275496, 1.9896435737609863),
 ('AMERICAN PIPIT', 0.41423888122034846, 1.8910620212554932),
 ('AMERICAN REDSTART', 0.07691659181313037, 1.9166347980499268),
 ('ANHINGA', 0.12015567965893287, 1.7394776344299316),
 ('ANNAS HUMMINGBIRD', 0.22766336012546198, 1.9383349418640137),
 ('ANTBIRD', 0.05559344960752011, 1.9913489818572998),
 ('ARARIPE MANAKIN', 0.515136974595801, 2.379765748977661),
 ('ASIAN CRESTED IBIS', 0.203816547964852, 2.0956268310546875),
 ('BA

In [ ]:
ap_at_n = np.array([ap[1] for ap in aps])
query_time = np.array(([ap[2] for ap in aps]))

In [ ]:
mAP_at_n = np.mean(ap_at_n, axis=0)
avg_query_time = np.mean(query_time, axis=0)
print("mAP:", mAP_at_n)
print("avg. query time: ", avg_query_time)

mAP: 0.29866201116286883
avg. query time:  1.9043744894174428


In [ ]:
save_results('/content/drive/MyDrive/CV_Birds/performance/fine_tuning/index/AutoEncoder', 'AE_FT_pert_tree_euclidean_results', aps)